# Introduction: Home Credit Default Risk Competition

# Data

The data is provided by [Home Credit](http://www.homecredit.net/about-us.aspx), a service dedicated to provided lines of credit
(loans) to the unbanked population. The goal is to predict if  client will repay a loan or have difficulty is a critical business need,




![image](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)


application_{train|test}.csv

    This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
    Static data for all applications. One row represents one loan in our data sample.

bureau.csv

    All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
    For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

bureau_balance.csv

    Monthly balances of previous credits in Credit Bureau.
    This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

POS_CASH_balance.csv

    Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
    This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

credit_card_balance.csv

    Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
    This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

previous_application.csv

    All previous applications for Home Credit loans of clients who have loans in our sample.
    There is one row for each previous application related to loans in our data sample.

installments_payments.csv

    Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
    There is a) one row for every payment that was made plus b) one row each for missed payment.
    One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

please check my kernel for simple exploration of all variables - https://www.kaggle.com/ganeshn88/simple-exploration-of-all-200-variables

# Loading Libraries


In [1]:
#numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

#Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

#Model building
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb

#Model Evaluation

from sklearn.metrics import roc_auc_score

#Saving Models
from sklearn.externals import joblib

In [2]:
# List files available
print(os.listdir("../input/home-credit-default-risk/"))

['previous_application.csv', 'installments_payments.csv', 'credit_card_balance.csv', 'application_train.csv', 'bureau.csv', 'application_test.csv', 'HomeCredit_columns_description.csv', 'bureau_balance.csv', 'POS_CASH_balance.csv', 'sample_submission.csv']


# Reducing Memory Usage

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

# Importing Data

In [4]:
def importdata():    
    print('-' * 80)
    print('train')
    train = import_data('../input/home-credit-default-risk/application_train.csv')

    print('-' * 80)
    print('test')
    test = import_data('../input/home-credit-default-risk/application_test.csv')

    print('-' * 80)
    print('bureau_balance')
    bureau_balance = import_data('../input/home-credit-default-risk/bureau_balance.csv')

    print('-' * 80)
    print('bureau')
    bureau = import_data('../input/home-credit-default-risk/bureau.csv')

    print('-' * 80)
    print('credit_card_balance')
    credit_card = import_data('../input/home-credit-default-risk/credit_card_balance.csv')

    print('-' * 80)
    print('installments_payments')
    installments = import_data('../input/home-credit-default-risk/installments_payments.csv')

    print('-' * 80)
    print('pos_cash_balance')
    pos_cash = import_data('../input/home-credit-default-risk/POS_CASH_balance.csv')

    print('-' * 80)
    print('previous_application')
    previous_app = import_data('../input/home-credit-default-risk/previous_application.csv')
    return train,test,bureau_balance,bureau,credit_card,installments,pos_cash,previous_app

# Handling Categorical data

## Label Encoding 

In [5]:
def encode_data(app_train,app_test):
    le = LabelEncoder()
    le_count = 0
    
    for col in app_train:
        if app_train[col].dtype == 'object':
            if len(list(app_train[col].unique())) <= 2:
                le.fit(app_train[col])
                app_train[col] = le.transform(app_train[col])
                app_test[col] = le.transform(app_test[col])
                le_count += 1
            
    print('%d columns were label encoded.' % le_count)

# one-hot encoding of categorical variables
    app_train = pd.get_dummies(app_train)
    app_test = pd.get_dummies(app_test)
    
    train_labels = app_train['TARGET']
    print('Aligning Train and Test Data')
    app_train, app_test = app_train.align(app_test, join='inner', axis=1)
    

    print('Training Features shape: ', app_train.shape)
    print('Testing Features shape: ', app_test.shape)
    
    return app_train,app_test,train_labels

def one_hot_code(app_train,app_test):
    
    app_train = pd.get_dummies(app_train)
    app_test = pd.get_dummies(app_test)
    
    train_labels = app_train['TARGET']
    print('Aligning Train and Test Data')
    app_train, app_test = app_train.align(app_test, join='inner', axis=1)
    

    print('Training Features shape: ', app_train.shape)
    print('Testing Features shape: ', app_test.shape)
    
    return app_train,app_test,train_labels
    

# Scaling Data

In [6]:
def scale_data(traindata,testdata):
   
    imputer = SimpleImputer(strategy = 'median')

    # Scale each feature to 0-1
    scaler = MinMaxScaler(feature_range = (0, 1))

    train = traindata.copy()
    test = testdata.copy()

    imputer.fit(train)
    train = imputer.transform(train)
    test = imputer.transform(test)

    scaler.fit(train)
    train = scaler.transform(train)
    test = scaler.transform(test)

    print('Training data shape: ', train.shape)
    print('Testing data shape: ', test.shape)

    return train,test


In [7]:
def featureengineering(df):
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

## Building Model

In [8]:
def preprocess_buildmodel(train,test,cvtype):
    print('--------Encoding Categorical Data----------')
    app_train_encoded,app_test_encoded,target = one_hot_code(train,test)
    print('-----------Scaling Data------------------------')
    train_scaled,test_scaled = scale_data(app_train_encoded,app_test_encoded)
    print('---------Splitting Data-----------------')
    X_train,X_test,Y_train,Y_test = train_test_split(train_scaled,target,test_size=0.2,random_state = 1)
    params = {
    'xgb': { 'learning_rate' : [0.05,0.10] ,'max_depth': [ 3,4,5,6,7] ,'n_estimators' : [500,1000] },
    'lgb': {  'learning_rate' : [0.05,0.10] ,'max_depth': [ 3,4,5,6,7] ,'n_estimators' : [500,1000]  }
    }
    models = {
    'xgb': XGBClassifier(), 
    'lgb':lgb.LGBMClassifier()
    }
    
    from collections import defaultdict
    model_result = defaultdict(list)
    
    for key in models.keys():
        
        model = models[key]
        param = params[key]
        
        if cvtype == 'grid':
            clf=GridSearchCV(model,param, cv=5, verbose=0)
        elif cvtype == 'rand':
            clf = RandomizedSearchCV(model, param, random_state=1, n_iter=100, cv=5, verbose=0, n_jobs=-1)
    
        clf.fit(X_train,Y_train,early_stopping_rounds=10,eval_metric = 'auc',eval_set=[(X_train, Y_train), (X_test, Y_test)])
    
        aucscoretr = roc_auc_score(Y_train, clf.predict_proba(X_train)[:,1])
        aucscore = roc_auc_score(Y_test, clf.predict_proba(X_test)[:,1])
        joblib_file = key + '_' + cvtype + '_model.pkl'
        joblib.dump(clf, joblib_file)
        model_result[key] = clf,clf.best_estimator_.get_params(),aucscoretr,aucscore,joblib_file
        
    return model_result,X_train,X_test,Y_train,Y_test,test_scaled
    

# Importing All data

In [9]:
train,test,bureau_balance,bureau,credit_card,installments,pos_cash,previous_app = importdata()

--------------------------------------------------------------------------------
train
Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 59.54 MB
Decreased by 79.2%
--------------------------------------------------------------------------------
test
Memory usage of dataframe is 45.00 MB
Memory usage after optimization is: 9.40 MB
Decreased by 79.1%
--------------------------------------------------------------------------------
bureau_balance
Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 156.21 MB
Decreased by 75.0%
--------------------------------------------------------------------------------
bureau
Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 78.57 MB
Decreased by 64.7%
--------------------------------------------------------------------------------
credit_card_balance
Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 263.69 MB
Decreased by 60.9%
---------------------------

# Building Model with only Application Data 

In [10]:
data_size = 100000

In [11]:
app_train = train.sample(data_size)
app_test = test
print('Appl data columns',train.columns)
print('Shape',app_train.shape)
model_result,X_train,X_test,Y_train,Y_test,test_scaled =preprocess_buildmodel(app_train,app_test,'rand')

Appl data columns Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)
Shape (100000, 122)
--------Encoding Categorical Data----------
Aligning Train and Test Data
Training Features shape:  (100000, 242)
Testing Features shape:  (48744, 242)
-----------Scaling Data------------------------
Training data shape:  (100000, 242)
Testing data shape:  (48744, 242)
---------Splitting Data-----------------
[0]	validation_0-auc:0.67762	validation_1-auc:0.67920
Multiple eval metrics have been passed: 'validation_1-auc' will be used for ea

In [12]:
print('XGB Train AUC with App data only',model_result['xgb'][2])
print('XGB Validation AUC with App data only',model_result['xgb'][3])
print('LGB Train AUC with App data only',model_result['lgb'][2])
print('LGB Validation AUC with App data only',model_result['lgb'][3])

XGB Train AUC with App data only 0.7803632330491814
XGB Validation AUC with App data only 0.7573292543809276
LGB Train AUC with App data only 0.8158656522318719
LGB Validation AUC with App data only 0.7571121936284059


In [13]:
lgb_model = joblib.load(model_result['lgb'][4])
xgb_model = joblib.load(model_result['xgb'][4])
lgb_pred = lgb_model.predict_proba(test_scaled)[:, 1]
xgb_pred = xgb_model.predict_proba(test_scaled)[:, 1]
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = (lgb_pred + xgb_pred)/2

In [14]:
#submit.to_csv('homecredit_withappdata_blend.csv', index = False)

## Combining Application data with Previous Application Data

In [15]:
prev_apps_avg = previous_app.groupby('SK_ID_CURR').mean()
prev_apps_avg.columns = ['p_' + col for col in prev_apps_avg.columns]
train_prev_app = train.merge(right=prev_apps_avg.reset_index(), how='left', on='SK_ID_CURR')

avg_inst = installments.groupby('SK_ID_CURR').mean()
avg_inst.columns = ['i_' + f_ for f_ in avg_inst.columns]
train_prev_app_inst = train_prev_app.merge(right=avg_inst.reset_index(), how='left', on='SK_ID_CURR')

pos_cash = pos_cash.groupby('SK_ID_CURR').mean()
train_prev_app_inst_pos = train_prev_app_inst.merge(right=pos_cash.reset_index(), how='left', on='SK_ID_CURR')

avg_cc_bal = credit_card.groupby('SK_ID_CURR').mean()
avg_cc_bal.columns = ['cc_bal_' + f_ for f_ in avg_cc_bal.columns]
train_prev_app_inst_pos_credit = train_prev_app_inst_pos.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')

bureau_avg = bureau.groupby('SK_ID_CURR').mean()
bureau_avg.columns = ['B_' + f_ for f_ in bureau_avg.columns]
train_prev_app_inst_pos_credit_bureau = train_prev_app_inst_pos_credit.merge(right=bureau_avg.reset_index(), how='left', on='SK_ID_CURR')

In [16]:
app_train = train_prev_app_inst_pos_credit_bureau.sample(data_size)
app_test = test
print('Shape',app_train.shape)
model_result,X_train,X_test,Y_train,Y_test,test_scaled =preprocess_buildmodel(app_train,app_test,'rand')

Shape (100000, 189)
--------Encoding Categorical Data----------
Aligning Train and Test Data
Training Features shape:  (100000, 242)
Testing Features shape:  (48744, 242)
-----------Scaling Data------------------------
Training data shape:  (100000, 242)
Testing data shape:  (48744, 242)
---------Splitting Data-----------------
[0]	validation_0-auc:0.73453	validation_1-auc:0.71471
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.74285	validation_1-auc:0.71962
[2]	validation_0-auc:0.74822	validation_1-auc:0.72061
[3]	validation_0-auc:0.75216	validation_1-auc:0.72405
[4]	validation_0-auc:0.75485	validation_1-auc:0.72372
[5]	validation_0-auc:0.75957	validation_1-auc:0.72674
[6]	validation_0-auc:0.76185	validation_1-auc:0.72813
[7]	validation_0-auc:0.76402	validation_1-auc:0.72840
[8]	validation_0-auc:0.76608	validation_1-auc:0.72920
[9]	validation_0-auc:0.7685

In [17]:
lgb_model = joblib.load(model_result['lgb'][4])
xgb_model = joblib.load(model_result['xgb'][4])
lgb_pred = lgb_model.predict_proba(test_scaled)[:, 1]
xgb_pred = xgb_model.predict_proba(test_scaled)[:, 1]
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = (lgb_pred + xgb_pred)/2

In [18]:
print('XGB Train AUC with Full data only',model_result['xgb'][2])
print('XGB Validation AUC with Full data only',model_result['xgb'][3])
print('LGB Train AUC with Full data only',model_result['lgb'][2])
print('LGB Validation AUC with Full data only',model_result['lgb'][3])

XGB Train AUC with Full data only 0.8608155278887751
XGB Validation AUC with Full data only 0.7511182381681556
LGB Train AUC with Full data only 0.7887950925900052
LGB Validation AUC with Full data only 0.7534468478159231


In [19]:
submit.to_csv('homecredit_withprevapp_blend.csv', index = False)

References:

Reducing Memory usage  - https://www.kaggle.com/gemartin/load-data-reduce-memory-usage